In [2]:
import os
import glob
import csv
import numpy as np
import re

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import numpy as np

#training Classification

twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)

text_clf = Pipeline([('vect', CountVectorizer()),('tfidf',TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None)),])
#text_clf = Pipeline([('vect', CountVectorizer()),('tfidf',TfidfTransformer()),('clf', MultinomialNB()),])

#Bayes Naive Train Model
text_clf.fit(twenty_train.data, twenty_train.target)  

twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)



/home/ubun/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
path = './TweetsData'
extension = 'csv'
os.chdir(path)
result = [i for i in glob.glob('*.{}'.format(extension))]
print (result)

['Twitter.csv', 'NASA.csv', 'rihanna.csv', 'MoneyUnder30.csv', 'ElonMusk.csv', 'Oprah.csv', 'cnnbrk.csv', 'jtimberlake.csv', 'HillaryClinton.csv', 'Official_Markfb.csv', 'taylorswift13.csv', 'TheEllenShow.csv', 'ladygaga.csv', 'shakira.csv', 'Cristiano.csv', 'realdonaldtrump.csv', 'barackobama.csv', 'jimmyfallon.csv', 'KylieJenner.csv', 'JeffBezos0147.csv', 'BillGates.csv', 'KimKardashian.csv', 'YouTube.csv', 'Ellen.csv', 'Yahoo.csv', 'katyperry.csv', 'JustinTrudeau.csv', 'justinbieber.csv', 'selenagomez.csv', 'ddlovato.csv', 'ArianaGrande.csv']


In [15]:
def removeSubString(s,sub):
    start = s.find(sub)
    end = s.find(' ', start)
    if (end ==-1):
        end = len(s)
    tmp = s[:start] + s[end:]
    if (tmp.find(sub)!=-1):
        tmp = removeSubString(tmp,sub)
    return (tmp)

def readFile(fname):
    ans = []
    with open(fname) as f:
        reader = csv.reader(f,delimiter=';')
        for row in reader:
            tmp = [row[0],row[1],row[2],row[3],row[4]]
            ans.append(tmp)
    return ans

def cleanString(s):
    tmp = removeSubString(s,"http")
    tmp = removeSubString(tmp,"@")
    tmp = removeSubString(tmp,"#")
    tmp = removeSubString(tmp,"&amp")
    tmp = re.sub('[^a-zA-Z ]+', ' ', tmp)
    tmp = re.sub( '\s+', ' ', tmp ).strip()
    return tmp.lower()

def getCategory(result):
    
    ans = []
    for f in result:
        ans = ans+readFile(f)

    data = np.array(ans)

    tweetL = data[:,4].tolist()
    tweetLClean=[]
    for i in tweetL:
        tmp = cleanString(i)
        tweetLClean.append(tmp)
    return data,text_clf.predict(tweetLClean)


twitter thanks all who shared on follow the hashtag to see stories behind the profession twitter com i moments


In [16]:
data,predicted = getCategory(result)


In [10]:
cat=twenty_train.target_names
tmp=[]
for i in predicted:
    tmp.append(cat[i])
predicted = tmp


In [11]:
predictedNP=np.transpose(np.array(predicted))


# In[9]:


final = np.column_stack((data, predictedNP))


# In[10]:

In [21]:
print((final[1][1]))

2018-05-04 03:43


In [8]:
from datetime import date
from datetime import datetime
def getWeek(d):
    StartDate = date(2016, 1, 4)
    end = d
    ans = int((end - StartDate).days/7)
    return ans 
def getDay(d):
    StartDate = date(2016, 1, 4)
    end = d
    ans = int((end - StartDate).days)
    return ans 
def convertToDate(s):
    return datetime.strptime(s,"%Y-%m-%d %H:%M").date()

weekList =[]

for i in final[:,1]:
    weekList.append(getDay(convertToDate(i)))

# In[13]:
# In[12]:

In [9]:
weekDay = np.transpose(np.array(weekList))
len(np.unique(weekDay))


841

In [10]:
numDay = (date(2018,5,4) - date(2016,1,4)).days
countTable = list(range(1,numDay))

In [12]:
catList = final[:,5]
cat = list(twenty_train.target_names)


# In[17]:


xMatrix = [[0 for x in range(len(cat))] for y in range(len(countTable))] 

#xMatrix is 20*Weeks, xMatrix[counTable.index(int)][cat.index(string)]


# In[18]:


count =0
for i in range(len(catList)):
    day = int(weekDay[i])
    c = catList[i]
    if (day not in countTable):
        continue;
    xMatrix[countTable.index(day)][cat.index(c)]+=1
   


# In[19]:


normMatrix=[]
for r in xMatrix:
    total = sum(r)
    if (total == 0):
        normMatrix.append(r)
        continue;
    tmp = [i/total for i in r]
    normMatrix.append(tmp)



In [31]:
path = '..'
os.chdir(path)
print(os.getcwd())
with open("/home/ubun/Desktop/StockPre/Stock/stock_data/fluctuations/finalFluctuation.txt") as f:
    content = f.readlines()
    content = [x.strip() for x in content] 

/home/ubun/Desktop/StockPre


In [32]:
sdf = re.split(':| ',content[0])
stocksName = [x.strip() for x in sdf[2::2]]


In [33]:
yMatrix = [[0 for x in range(len(countTable))] for y in range(len(stocksName))] 
#[-1,0,1] with -1 is loss, 0 is neutral, 1 is gain


# In[ ]:

In [35]:
len(countTable)

850

In [36]:
inputEntries = []
for x in content:
    inputEntries.append(int(x.split(":")[0].split(" ")[1])-2)
inputEntries.pop(0)
inputEntries.pop()

806

In [37]:
threshold = 2.0
try: 
    for i in range(len(content)):
        a = re.split(':| ',content[i])
        tmp = [x.strip() for x in a[3::2]]
        for j in range(len(tmp)):
            s = re.sub('[^0-9-.]+', '',tmp[j])
            if (s == "" or s=="NAN"):
                n = 0.0
            else: 
                n = float(s)
            if (n<=(-1*threshold)):
                yMatrix[j][i]= "Loss"
            elif (n>=threshold):
                yMatrix[j][i]= "Gain"
            else:
                yMatrix[j][i]= "Neutral"
except (IndexError):
    print (i)
    print (j)


In [42]:
#delete Weekend
#remove last row
inputs = np.delete(xMatrix,(len(xMatrix)-1),axis=0)
#remove first column
outputs = np.delete(yMatrix,0,axis=1)

In [43]:


#take relevant inputs
yTemp=np.transpose(outputs)
fInput=[]
fOutPut = []
for i in inputEntries:
    fInput.append(inputs[i])
    fOutPut.append(yTemp[i])


# In[27]:


fOutPut = np.transpose(np.array(fOutPut))
fInput = np.array(fInput)


# In[28]:


import pickle
with open('objs.pkl', 'wb') as f: 
    pickle.dump([fOutPut, fInput,stocksName], f)
f.close()


# In[29]:


import pickle
with open('objs.pkl','rb') as f:  # Python 3: open(..., 'rb')
    fOutPut, fInput,stocksName = pickle.load(f)
f.close()



/home/ubun/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


--- 20.460599660873413 Secs ---


In [54]:

#building SVM for each stock
#row
from sklearn.model_selection import cross_val_score
from sklearn import svm
import time
start_time = time.time()

index = -1
pickedStock = ["BDX","GOOGL","PGR","JNJ","CSCO","CLX","PG","MSFT","CHKP","ORCL","DIS","SAFT","ITC",
              "KO","WMT","VZ","TRV","MCD","COST"]
for stock in fOutPut:
    index +=1
    if (stocksName[index] not in pickedStock):
        continue;
        
    clf = svm.SVC(decision_function_shape='ovo',kernel='sigmoid')
    scores = cross_val_score(clf, fInput, stock, cv=5)
    #print stock Name
    
    file.write (stocksName[index]+"\n")
    file.write("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)+"\n")
    file.write ("\n")
file.close() 
print("--- %s Secs ---" % (time.time() - start_time))
#working up till here


# In[ ]
path = './Stock'
os.chdir(path)


/home/ubun/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


--- 1.3089878559112549 Secs ---


In [82]:

import Exporter
import os.path
import got3
import codecs
from threading import Thread
from datetime import timedelta
from datetime import datetime
start_time = time.time()
def getData(name,startObj,endObj):
#     print ("getting Data")
    start = startObj.strftime('%Y-%m-%d')
    end = endObj.strftime('%Y-%m-%d')
    tweetCriteria = got3.manager.TweetCriteria().setUsername(name).setSince(start).setUntil(end).setMaxTweets(0)
    tweet = got3.manager.TweetManager.getTweets(tweetCriteria)

    outputFileName = "Prediction/%s.csv"%(name)
    outputFile = codecs.open(outputFileName, "a", "utf-8")
    for t in tweet:
        outputFile.write(('%s;%s;%d;%d;"%s"\n' % (outputFileName,t.date.strftime("%Y-%m-%d %H:%M"), t.retweets, t.favorites, t.text)))
    outputFile.flush()

ppl=["BillGates","realdonaldtrump","Ellen","ElonMusk","barackobama","katyperry",'justinbieber','rihanna',
    'ladygaga','taylorswift13','TheEllenShow','Cristiano','YouTube','jtimberlake','Twitter','KimKardashian',
    'ArianaGrande','ddlovato','selenagomez','cnnbrk','shakira','jimmyfallon','HillaryClinton',
    'JeffBezos0147','JustinTrudeau','KylieJenner','Official_Markfb','MoneyUnder30','NASA','Oprah','Yahoo']


threads=[]
for p in ppl:
    t = Thread(target = getData, args = (p,date(2018,5,6),date(2018,5,7),)) 
    threads.append(t)
for t in threads:
    t.start()
for t in threads:
    t.join()
print("Exit main thread")
print("--- %s seconds ---" % (time.time() - start_time))


Exit main thread
--- 3.5818567276000977 seconds ---


In [83]:
path = './Prediction'
extension = 'csv'
os.chdir(path)
result = [i for i in glob.glob('*.{}'.format(extension))]
print (result)

['Twitter.csv', 'NASA.csv', 'rihanna.csv', 'MoneyUnder30.csv', 'ElonMusk.csv', 'Oprah.csv', 'cnnbrk.csv', 'jtimberlake.csv', 'HillaryClinton.csv', 'Official_Markfb.csv', 'taylorswift13.csv', 'TheEllenShow.csv', 'ladygaga.csv', 'shakira.csv', 'Cristiano.csv', 'realdonaldtrump.csv', 'barackobama.csv', 'jimmyfallon.csv', 'KylieJenner.csv', 'JeffBezos0147.csv', 'BillGates.csv', 'KimKardashian.csv', 'YouTube.csv', 'Ellen.csv', 'Yahoo.csv', 'katyperry.csv', 'JustinTrudeau.csv', 'justinbieber.csv', 'selenagomez.csv', 'ddlovato.csv', 'ArianaGrande.csv']
